# Embætti landlæknis gögn

In [1]:
import numpy as np 
import pandas as pd
import plotly.graph_objects as go
import csv

#xls = pd.ExcelFile('legurogkomur.xlsx')
df = pd.read_csv('Komur per dag.csv')

In [2]:
df2 = pd.read_csv("Legur per dag.csv")
df2["Dagsetning innritunar"] = pd.to_datetime(df2["Dagsetning innritunar"])
df2["Dagsetning útskriftar"] = pd.to_datetime(df2["Dagsetning útskriftar"])
df2 = df2[df2['Dagsetning innritunar'].dt.year != 2009]
df2


,Starfstöð,Ár útskriftar,Dagsetning innritunar,Dagsetning útskriftar,Legudagar,Kom frá - nr,Fór til - nr,Tilvísun - nr,Kyn,Aldursflokkur
425,Landspítali,2010,2010-01-01,2010-01-01,1,6.0,1.0,0.0,Karl,0-49 ára
426,Landspítali,2010,2010-01-01,2010-01-01,1,6.0,1.0,0.0,Kona,0-49 ára
427,Landspítali,2010,2010-01-01,2010-01-01,1,8.0,1.0,9.0,Kona,0-49 ára
428,Landspítali,2010,2010-01-01,2010-02-01,1,1.0,1.0,0.0,Karl,50-69 ára
429,Landspítali,2010,2010-01-01,2010-02-01,2,1.0,1.0,0.0,Kona,0-49 ára
...,...,...,...,...,...,...,...,...,...,...
482219,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-16,2022-12-23,7,3.0,1.0,NaN,Kona,70 ára og eldri
482220,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-20,2022-12-21,1,1.0,1.0,8.0,Karl,0-49 ára
482221,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-21,2022-12-24,3,1.0,1.0,8.0,Kona,70 ára og eldri
482222,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-22,2022-12-23,1,1.0,3.0,8.0,Karl,70 ára og eldri


In [3]:
date_counts = df2["Dagsetning útskriftar"].value_counts()

# Create a date range from the minimum to the maximum date in the column
date_range = pd.date_range(start=df2['Dagsetning útskriftar'].min(), end=df2['Dagsetning útskriftar'].max(), freq='D')

# Reindex the date_counts Series with the full date range and fill missing values with 0
date_counts = date_counts.reindex(date_range, fill_value=0)

# Convert the Series to a list
kut = date_counts.tolist()

date_counts = df2['Dagsetning innritunar'].value_counts()

# Create a date range from the minimum to the maximum date in the column
date_range = pd.date_range(start=df2['Dagsetning innritunar'].min(), end=df2['Dagsetning innritunar'].max(), freq='D')

# Reindex the date_counts Series with the full date range and fill missing values with 0
date_counts = date_counts.reindex(date_range, fill_value=0)

# Convert the Series to a list
kinn = date_counts.tolist()

In [4]:
dagar = []
dagar.append(kinn[0]-kut[0])
for i in range(1,len(kinn)):
    dagar.append(kinn[i]+dagar[i-1]-kut[i])

print(len(dagar))
fig = go.Figure(data=[go.Histogram(x=dagar)])
fig.update_layout(title='Dreifing daga',
                  xaxis=dict(title='Dagar'),
                  yaxis=dict(title='Tíðni'),)
fig.show() # (e.g. það eru 346 dagar þar sem 1100 manns eru inni)

df_ticks = pd.DataFrame({'x': range(0, 5001, 365), 'date': pd.date_range(start='2010-01-01', periods=14, freq='365D')})


import plotly.express as px
plot = px.line(dagar)
plot.update_layout(xaxis=dict(title="Dagsetning",
                              tickmode='array',
                              tickvals=df_ticks['x'],
                              ticktext=df_ticks['date'].dt.strftime('%d.%m.%Y')),
                   yaxis=dict(title="Fjöldi per dag"),
                   title="Inniliggjandi sjúklingar 2010-2022")
plot.show() 

4748


In [5]:
with open("dagar.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Fjöldi á dag'])  # Optional: Write header row if needed
    writer.writerows(zip(dagar))

In [22]:
dagar = dagar[250:4251] # 08.09.2010 - 21.08.2021

In [23]:
p = px.line(dagar)
df_ticks = pd.DataFrame({'x': range(0, 4253, 365), 'date': pd.date_range(start='2010-09-08', periods=12, freq='365D')})
p.update_layout(xaxis=dict(title="Dagsetning",
                              tickmode='array',
                              tickvals=df_ticks['x'],
                              ticktext=df_ticks['date'].dt.strftime('%d.%m.%Y')),
                   yaxis=dict(title="Fjöldi per dag"),
                   title="Inniliggjandi sjúklingar 08.09.10-21.08.21")
p.show()

In [34]:

grouped_df = df2.groupby(['Dagsetning innritunar', 'Aldursflokkur']).size().reset_index(name='Count')

mean_df = grouped_df.groupby('Aldursflokkur')['Count'].mean().reset_index(name='Meðal Count')

mean_df

,Aldursflokkur,Meðal Count
0,0-49 ára,43.739890
1,50-69 ára,23.198020
2,70 ára og eldri,34.490733


In [64]:
import plotly.express as px
grouped_data = df.groupby(['Starfstöð', 'Aldursflokkun komu']).size().reset_index(name='Fjöldi')

fig = px.bar(grouped_data, x='Starfstöð', y='Fjöldi', color='Aldursflokkun komu', barmode='group',
             title='Fjöldi koma eftir aldurshópi og starfstöð')

fig.show()

In [7]:
import plotly.express as px
import pandas as pd

# Kom frá (Keyra cell aðeins einu sinni, aðeins verður bar plot vitlaust)
value_mapping = {
    0: "Sjúkrahús erlendis",
    1: "Heimili (þ.m.t. þjónustuíbúðir)",
    2: "Sama sjúkrahús, önnur sérgrein",
    3: "Annað sjúkrahús",
    4: "4: Óskilgreint gildi",
    5: "5: Óskilgreint gildi",
    6: "Fædd(ur) hér",
    7: "Sjálfstætt starfandi læknastofa/-stöð",
    8: "Annað",
    9: "Óþekkt",
    10: "Hjúkrunar- og/eða dvalarheimili",
    11: "Sjúkrahótel"
}

df["Kom frá - nr"] = df["Kom frá - nr"].map(value_mapping)
df["Kom frá - nr"].fillna("Finnst ekki",inplace=True)
value_counts = df["Kom frá - nr"].value_counts()

In [8]:
df_value_counts = pd.DataFrame({"Gildi": value_counts.index, "Fjöldi": value_counts.values})
df_value_counts = df_value_counts.sort_values(by="Fjöldi", ascending=False)

total_count = value_counts.sum()
df_value_counts["Hlutfall"] = (df_value_counts["Fjöldi"] / total_count) * 100

fig = px.bar(df_value_counts, x="Gildi", y="Fjöldi", text="Hlutfall", title="Kom frá (Komur per dag)")
fig.update_traces(texttemplate='%{text:.2f}', textposition='auto')

fig.show()

In [9]:
# Fór til (Keyra cell einu sinni, annars verður bar plot vitlaust)

value_mapping = {
    0: "Sjúkrahús erlendis",
    1: "1: Óskilgreint gildi",
    2: "2: Óskilgreint gildi",
    3: "Annað sjúkrahús",
    4: "4: Óskilgreint gildi",
    5: "5: Óskilgreint gildi",
    6: "Andlát",
    7: "Sjúklingur útskrifaði sig sjálfur",
    8: "Mætti ekki (eingöngu fyrir ferilsjúklinga)",
    9: "Óþekkt",
    10: "Hjúkrunar- og/eða dvalarheimili",
    11: "Sjúkrahótel"
}

df["Fór til - nr"] = df["Fór til - nr"].map(value_mapping)

In [10]:
df["Fór til - nr"].fillna("Finnst ekki", inplace=True)

In [11]:
value_counts = df["Fór til - nr"].value_counts(dropna=False)
df_value_counts = pd.DataFrame({"Gildi": value_counts.index, "Fjöldi": value_counts.values})
df_value_counts = df_value_counts.sort_values(by="Fjöldi", ascending=False)

total_count = value_counts.sum()
df_value_counts["Hlutfall"] = (df_value_counts["Fjöldi"] / total_count) * 100

fig = px.bar(df_value_counts, x="Gildi", y="Fjöldi", text="Hlutfall", title="Fór til (Komur per dag)") 
fig.update_traces(texttemplate='%{text:.2f}', textposition='auto')

fig.show()

### Meðalfjöldi koma aðeins fyrir árið 2018

In [12]:
filtered_df = df[df['Ár komu'] == 2018]
filtered_df

,Starfstöð,Ár komu,Dagsetning komu,Kom frá - nr,Fór til - nr,Tilvísun - nr,Kyn,Aldursflokkun komu
187094,Landspítali,2018.0,01.01.2018,Heimili (þ.m.t. þjónustuíbúðir),1: Óskilgreint gildi,0.0,Karl,0-49 ára
187095,Landspítali,2018.0,01.01.2018,Heimili (þ.m.t. þjónustuíbúðir),1: Óskilgreint gildi,0.0,Karl,50-69 ára
187096,Landspítali,2018.0,01.01.2018,Heimili (þ.m.t. þjónustuíbúðir),1: Óskilgreint gildi,0.0,Karl,70 ára og eldri
187097,Landspítali,2018.0,01.01.2018,Heimili (þ.m.t. þjónustuíbúðir),1: Óskilgreint gildi,0.0,Kona,0-49 ára
187098,Landspítali,2018.0,01.01.2018,Heimili (þ.m.t. þjónustuíbúðir),1: Óskilgreint gildi,0.0,Kona,50-69 ára
...,...,...,...,...,...,...,...,...
641206,Sjúkrarými Stykkishólmi - HVE,2018.0,22.12.2018,Heimili (þ.m.t. þjónustuíbúðir),Finnst ekki,0.0,Kona,0-49 ára
641207,Sjúkrarými Stykkishólmi - HVE,2018.0,23.12.2018,Heimili (þ.m.t. þjónustuíbúðir),Finnst ekki,0.0,Kona,0-49 ára
641208,Sjúkrarými Stykkishólmi - HVE,2018.0,25.12.2018,Heimili (þ.m.t. þjónustuíbúðir),Finnst ekki,0.0,Karl,50-69 ára
641209,Sjúkrarými Stykkishólmi - HVE,2018.0,27.12.2018,Heimili (þ.m.t. þjónustuíbúðir),Finnst ekki,1.0,Karl,50-69 ára


In [3]:
df = filtered_df
# Fór til (Keyra cell einu sinni, annars verður bar plot vitlaust)

value_mapping = {
    0: "Sjúkrahús erlendis",
    1: "1: Óskilgreint gildi",
    2: "2: Óskilgreint gildi",
    3: "Annað sjúkrahús",
    4: "4: Óskilgreint gildi",
    5: "5: Óskilgreint gildi",
    6: "Andlát",
    7: "Sjúklingur útskrifaði sig sjálfur",
    8: "Mætti ekki (eingöngu fyrir ferilsjúklinga)",
    9: "Óþekkt",
    10: "Hjúkrunar- og/eða dvalarheimili",
    11: "Sjúkrahótel"
}

df["Fór til - nr"] = df["Fór til - nr"].map(value_mapping)

In [4]:
# Kom frá (Keyra cell aðeins einu sinni, aðeins verður bar plot vitlaust)
value_mapping = {
    0: "Sjúkrahús erlendis",
    1: "Heimili (þ.m.t. þjónustuíbúðir)",
    2: "Sama sjúkrahús, önnur sérgrein",
    3: "Annað sjúkrahús",
    4: "4: Óskilgreint gildi",
    5: "5: Óskilgreint gildi",
    6: "Fædd(ur) hér",
    7: "Sjálfstætt starfandi læknastofa/-stöð",
    8: "Annað",
    9: "Óþekkt",
    10: "Hjúkrunar- og/eða dvalarheimili",
    11: "Sjúkrahótel"
}

df["Kom frá - nr"] = df["Kom frá - nr"].map(value_mapping)
df["Kom frá - nr"].fillna("Finnst ekki",inplace=True)
value_counts = df["Kom frá - nr"].value_counts()

In [13]:
import plotly.express as px
mean_visits = df.groupby(['Dagsetning komu', 'Aldursflokkun komu']).size().unstack().mean()
mean_visits

grouped_df = df.groupby(['Dagsetning komu', 'Aldursflokkun komu']).size().reset_index(name='Fjöldi koma')

fig = px.box(grouped_df, x='Aldursflokkun komu', y='Fjöldi koma')

fig.update_layout(xaxis_title='Aldurshópur', yaxis_title='Vísiteringar',title="Dreifing [Komur per dag] fyrir árið 2018")

fig.show()

## Legur per dag

In [67]:
df2
df2 = df2.dropna(subset=["Aldursflokkur"])
df2

,Starfstöð,Ár útskriftar,Dagsetning innritunar,Dagsetning útskriftar,Legudagar,Kom frá - nr,Fór til - nr,Tilvísun - nr,Kyn,Aldursflokkur
425,Landspítali,2010,2010-01-01,2010-01-01,1,6.0,1.0,0.0,Karl,0-49 ára
426,Landspítali,2010,2010-01-01,2010-01-01,1,6.0,1.0,0.0,Kona,0-49 ára
427,Landspítali,2010,2010-01-01,2010-01-01,1,8.0,1.0,9.0,Kona,0-49 ára
428,Landspítali,2010,2010-01-01,2010-02-01,1,1.0,1.0,0.0,Karl,50-69 ára
429,Landspítali,2010,2010-01-01,2010-02-01,2,1.0,1.0,0.0,Kona,0-49 ára
...,...,...,...,...,...,...,...,...,...,...
482219,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-16,2022-12-23,7,3.0,1.0,NaN,Kona,70 ára og eldri
482220,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-20,2022-12-21,1,1.0,1.0,8.0,Karl,0-49 ára
482221,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-21,2022-12-24,3,1.0,1.0,8.0,Kona,70 ára og eldri
482222,Sjúkrarými Stykkishólmi - HVE,2022,2022-12-22,2022-12-23,1,1.0,3.0,8.0,Karl,70 ára og eldri


In [77]:
import plotly.graph_objects as go
import numpy as np

ungurlog = np.log(df2[(df2["Legudagar"] <= 30) & (df2["Aldursflokkur"] == "0-49 ára")]["Legudagar"])
ungurmu = np.sum(ungurlog)/len(ungurlog)
ungurstd = np.sqrt(np.sum([(x-ungurmu)**2 for x in ungurlog])/len(ungurlog))

midlog = np.log(df2[(df2["Legudagar"] <= 30) & (df2["Aldursflokkur"] == "50-69 ára")]["Legudagar"])
midmu = np.sum(midlog)/len(midlog)
midstd = np.sqrt(np.sum([(x-midmu)**2 for x in midlog])/len(midlog))

oldlog = np.log(df2[(df2["Legudagar"] <= 30) & (df2["Aldursflokkur"] == "70 ára og eldri")]["Legudagar"])
oldmu = np.sum(oldlog)/len(oldlog)
oldstd = np.sqrt(np.sum([(x-oldmu)**2 for x in oldlog])/len(oldlog))

print(ungurmu,ungurstd)
print(midmu,midstd)
print(oldmu,oldstd)


0.9163059078519854 0.8444342744931783
1.2384774562570335 0.9410482416853067
1.5546870190466466 0.9770730458761157


### 2018

In [17]:
df2["Dagsetning innritunar"] = pd.to_datetime(df2["Dagsetning innritunar"])
df2["Dagsetning útskriftar"] = pd.to_datetime(df2["Dagsetning útskriftar"])
filtered_df = df2[df2['Dagsetning innritunar'].dt.year == 2018]
filtered_df

,Starfstöð,Ár útskriftar,Dagsetning innritunar,Dagsetning útskriftar,Legudagar,Kom frá - nr,Fór til - nr,Tilvísun - nr,Kyn,Aldursflokkur
196424,Landspítali,2018,2018-01-01,2018-01-01,1,1.0,1.0,0.0,Kona,0-49 ára
196425,Landspítali,2018,2018-01-01,2018-01-01,1,1.0,1.0,3.0,Kona,0-49 ára
196426,Landspítali,2018,2018-01-01,2018-01-01,1,1.0,1.0,8.0,Kona,0-49 ára
196427,Landspítali,2018,2018-01-01,2018-01-01,1,1.0,1.0,9.0,Karl,0-49 ára
196428,Landspítali,2018,2018-01-01,2018-01-01,1,2.0,1.0,NaN,Karl,0-49 ára
...,...,...,...,...,...,...,...,...,...,...
481738,Sjúkrarými Stykkishólmi - HVE,2018,2018-12-16,2018-12-21,5,1.0,1.0,1.0,Kona,0-49 ára
481739,Sjúkrarými Stykkishólmi - HVE,2018,2018-12-20,2018-12-22,2,3.0,1.0,NaN,Kona,70 ára og eldri
481740,Sjúkrarými Stykkishólmi - HVE,2018,2018-12-26,2018-12-27,1,10.0,10.0,NaN,Karl,50-69 ára
481741,Sjúkrarými Stykkishólmi - HVE,2019,2018-12-12,2019-01-18,37,3.0,1.0,NaN,Kona,50-69 ára


In [18]:
df = filtered_df
# Fór til (Keyra cell einu sinni, annars verður bar plot vitlaust)

value_mapping = {
    0: "Sjúkrahús erlendis",
    1: "1: Óskilgreint gildi",
    2: "2: Óskilgreint gildi",
    3: "Annað sjúkrahús",
    4: "4: Óskilgreint gildi",
    5: "5: Óskilgreint gildi",
    6: "Andlát",
    7: "Sjúklingur útskrifaði sig sjálfur",
    8: "Mætti ekki (eingöngu fyrir ferilsjúklinga)",
    9: "Óþekkt",
    10: "Hjúkrunar- og/eða dvalarheimili",
    11: "Sjúkrahótel"
}

df["Fór til - nr"] = df["Fór til - nr"].map(value_mapping)

<ipython-input-18-511ed6eca3e9>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
# Kom frá (Keyra cell aðeins einu sinni, aðeins verður bar plot vitlaust)
value_mapping = {
    0: "Sjúkrahús erlendis",
    1: "Heimili (þ.m.t. þjónustuíbúðir)",
    2: "Sama sjúkrahús, önnur sérgrein",
    3: "Annað sjúkrahús",
    4: "4: Óskilgreint gildi",
    5: "5: Óskilgreint gildi",
    6: "Fædd(ur) hér",
    7: "Sjálfstætt starfandi læknastofa/-stöð",
    8: "Annað",
    9: "Óþekkt",
    10: "Hjúkrunar- og/eða dvalarheimili",
    11: "Sjúkrahótel"
}

df["Kom frá - nr"] = df["Kom frá - nr"].map(value_mapping)
df["Kom frá - nr"].fillna("Finnst ekki",inplace=True)
value_counts = df["Kom frá - nr"].value_counts()

<ipython-input-19-5cfdf8614929>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\krist\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
grouped_df = df.groupby(['Dagsetning innritunar', 'Aldursflokkur']).size().reset_index(name='Fjöldi koma')

fig = px.box(grouped_df, x='Aldursflokkur', y='Fjöldi koma')

fig.update_layout(xaxis_title='Aldurshópur', yaxis_title='Vísiteringar',title="Dreifing [Legur per dag] fyrir árið 2018")

fig.show() # Tekið saman fjöldi fólks sem var innritað á hverjum degi
           # Ekki er tekið tillit til legutíma